# Lesson 3: Vectorstores and embeddings

# Vectorstore ingestion

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings();

await embeddings.embedQuery("This is some sample text");

[
   -0.010454769,   0.0023495338, -0.0008236494,  -0.010953553,    -0.01150484,
    0.022891548,   -0.014687868,  0.0017490244,  -0.017601814,   -0.019386934,
    0.005135503,    0.034127306,  -0.012298956,  0.0019163794,    0.004646564,
    0.013079946,    0.024689794,   0.001944272,  0.0044529573,  -0.0063594924,
  -0.0051158145, -0.00077852915,  -0.008078983,   0.014005321,   -0.009076551,
   -0.003957455,  -0.0008121643,  -0.019701956,    0.00401324,   -0.001540651,
    0.015934827,   -0.021972734,  -0.000726846,  -0.022261504,    0.006261048,
    0.007081416,   -0.011340766, -0.0135459155,    0.02871944,   -0.023469085,
   0.0069173425,    0.017641192,   0.017129282, -0.0059525897,  -0.0037868184,
   -0.014425349,   -0.001721132, -0.0060740043,   -0.00796085,    0.010533525,
    0.032972228,  -0.0008244698,  -0.024033498, -0.0034324194, -0.00075350795,
    0.002628459,   -0.022458393,   0.025227955,    0.01968883,    0.010894486,
  -0.0117542315,  -0.0025661108,  0.0025086852, -0

In [3]:
import { similarity } from "ml-distance";

const vector1 = await embeddings.embedQuery(
    "What are vectors useful for in machine learning?"
);
const unrelatedVector = await embeddings.embedQuery(
    "A group of parrots is called a pandemonium."
);

In [4]:
similarity.cosine(vector1, unrelatedVector);

0.6962144676957391

In [5]:
const similarVector = await embeddings.embedQuery(
    "Vectors are representations of information."
);

similarity.cosine(vector1, similarVector);

0.859059927625929

In [6]:
// Peer dependency
import * as parse from "pdf-parse";
import { PDFLoader } from "langchain/document_loaders/fs/pdf";
import { 
    RecursiveCharacterTextSplitter
} from "langchain/text_splitter";

const loader = new PDFLoader("./data/MachineLearning-Lecture01.pdf");

const rawCS229Docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 128,
  chunkOverlap: 0,
});

const splitDocs = await splitter.splitDocuments(rawCS229Docs);

In [7]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);

In [8]:
await vectorstore.addDocuments(splitDocs);

In [9]:
const retrievedDocs = await vectorstore.similaritySearch(
    "What is deep learning?", 
    4
);

const pageContents = retrievedDocs.map(doc => doc.pageContent);

pageContents

[
  "piece of research in machine learning, okay?",
  "are using a learning algorithm, perhaps without even being aware of it.",
  "some of my own excitement about machine learning to you.",
  "of the class, and then we'll start to talk a bit about machine learning."
]

# Retrievers

In [10]:
const retriever = vectorstore.asRetriever();

In [11]:
await retriever.invoke("What is deep learning?")

[
  Document {
    pageContent: "piece of research in machine learning, okay?",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 8, lines: { from: 2, to: 2 } }
    }
  },
  Document {
    pageContent: "are using a learning algorithm, perhaps without even being aware of it.",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",